# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key


In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,2.01,84,100,3.09,GB,1709010487
1,1,port-aux-francais,-49.3500,70.2167,6.07,69,4,5.32,TF,1709010488
2,2,college,64.8569,-147.8028,-13.82,54,75,2.06,US,1709010488
3,3,grytviken,-54.2811,-36.5092,-0.06,83,48,3.87,GS,1709010489
4,4,bethel,41.3712,-73.4140,5.18,84,0,1.54,US,1709010489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.



In [6]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot_1

c:\Users\pku11\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 21
ideal_temp_max = 27
ideal_wind_speed_max = 4.5
ideal_cloudiness = 0
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_temp_min) &
    (city_data_df["Max Temp"] <= ideal_temp_max) &
    (city_data_df["Wind Speed"] < ideal_wind_speed_max) &
    (city_data_df["Cloudiness"] == ideal_cloudiness)
]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
128,128,san luis de la loma,17.2706,-100.8939,23.68,83,0,1.94,MX,1709010538
177,177,abilene,32.4487,-99.7331,23.49,33,0,3.09,US,1709010557
188,188,rabigh,22.7986,39.0349,22.34,48,0,3.06,SA,1709010561
280,280,ambovombe,-25.1667,46.0833,26.86,71,0,2.35,MG,1709010596
309,309,santana,-0.0583,-51.1817,26.10,89,0,1.54,BR,1709010609
572,572,mazatlan,23.2167,-106.4167,21.06,89,0,0.00,MX,1709010610


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df.drop(['Max Temp', 'Wind Speed', 'Cloudiness', 'Date'], axis=1)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
0,0,stanley,54.8680,-1.6985,84,GB,
1,1,port-aux-francais,-49.3500,70.2167,69,TF,
2,2,college,64.8569,-147.8028,54,US,
3,3,grytviken,-54.2811,-36.5092,83,GS,
4,4,bethel,41.3712,-73.4140,84,US,
...,...,...,...,...,...,...,...
574,574,kyaikkami,16.0800,97.5675,55,MM,
575,575,dolinsk,47.3500,142.8000,83,RU,
576,576,severo-yeniseyskiy,60.3725,93.0408,85,RU,
577,577,rabaul,-4.1967,152.1721,72,PG,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
categories = "lodging"
radius = 10000
limit = 1
params = {
    "radius": 10000,
    "categories": categories,
    "apiKey": geoapify_key
    "limit":limit

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["Lat"] = latitude
    params["Lng"] = longitude

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel_data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
stanley - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
college - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
aitape - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
malaya vishera - nearest hotel: No hotel found
new plymouth - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
copainala - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
nyurba - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
carauari - nearest hotel: No hotel found
ceeldheer - nearest hotel: No hotel found
ataq - nearest hotel: No hotel found
horta - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
atafu village 

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
0,0,stanley,54.8680,-1.6985,84,GB,No hotel found
1,1,port-aux-francais,-49.3500,70.2167,69,TF,No hotel found
2,2,college,64.8569,-147.8028,54,US,No hotel found
3,3,grytviken,-54.2811,-36.5092,83,GS,No hotel found
4,4,bethel,41.3712,-73.4140,84,US,No hotel found
...,...,...,...,...,...,...,...
574,574,kyaikkami,16.0800,97.5675,55,MM,No hotel found
575,575,dolinsk,47.3500,142.8000,83,RU,No hotel found
576,576,severo-yeniseyskiy,60.3725,93.0408,85,RU,No hotel found
577,577,rabaul,-4.1967,152.1721,72,PG,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)